First of All let us import the dataset into an environment for processing and rectifying

Import library files needed for the task

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel("/content/2019 Winter Data Science Intern Challenge Data Set.xlsx")

upload the dataset and see a glimse of it

In [ ]:
df

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1.0,53.0,746.0,224.0,2.0,cash,2017-03-13 12:36:56.190
1,2.0,92.0,925.0,90.0,1.0,cash,2017-03-03 17:38:51.999
2,3.0,44.0,861.0,144.0,1.0,cash,2017-03-14 04:23:55.595
3,4.0,18.0,935.0,156.0,1.0,credit_card,2017-03-26 12:43:36.649
4,5.0,18.0,883.0,156.0,1.0,credit_card,2017-03-01 04:35:10.773
...,...,...,...,...,...,...,...
4995,4996.0,73.0,993.0,330.0,2.0,debit,2017-03-30 13:47:16.597
4996,4997.0,48.0,789.0,234.0,2.0,cash,2017-03-16 20:36:16.389
4997,4998.0,56.0,867.0,351.0,3.0,cash,2017-03-19 05:42:42.228
4998,4999.0,60.0,825.0,354.0,2.0,credit_card,2017-03-16 14:51:18.188


Let us look into the summary of the dataset

In [ ]:
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


Since the orders are at bulk in few shops so let us find the price per item in the shops and check the values of the price of shoes

In [ ]:
df["price_per_item"]=df['order_amount']/df['total_items']
print(df.describe())
print("max :", df['price_per_item'].max(),"min :", df['price_per_item'].min())
print("mean :",df['price_per_item'].mean(),"median :",df['price_per_item'].median())


          order_id      shop_id      user_id   order_amount  total_items  \
count  5000.000000  5000.000000  5000.000000    5000.000000   5000.00000   
mean   2500.500000    50.078800   849.092400    3145.128000      8.78720   
std    1443.520003    29.006118    87.798982   41282.539349    116.32032   
min       1.000000     1.000000   607.000000      90.000000      1.00000   
25%    1250.750000    24.000000   775.000000     163.000000      1.00000   
50%    2500.500000    50.000000   849.000000     284.000000      2.00000   
75%    3750.250000    75.000000   925.000000     390.000000      3.00000   
max    5000.000000   100.000000   999.000000  704000.000000   2000.00000   

       price_per_item  
count     5000.000000  
mean       387.742800  
std       2441.963725  
min         90.000000  
25%        133.000000  
50%        153.000000  
75%        169.000000  
max      25725.000000  
max : 25725.0 min : 90.0
mean : 387.7428 median : 153.0


Looking into the maximum and minimum value of shoes we find that a particular shoe is very costly. Since the shoes are very affordable Then there must be shoe prices which are outliers(very far from central value)

Let us create a new dataframe having prices per item and shop id. 
Using drop duplicates since there may be multiple order from a shop and since each shoe cost the same in same shop the value may be redundant.

In [42]:
check_high_low=df[["shop_id","price_per_item"]].copy().drop_duplicates()

print(check_high_low.nlargest(10,"price_per_item"))
print(check_high_low.nsmallest(10,"price_per_item"))


     shop_id  price_per_item
160     78.0         25725.0
15      42.0           352.0
107     12.0           201.0
205     89.0           196.0
44      99.0           195.0
90      50.0           193.0
242     38.0           190.0
55      51.0           187.0
116      6.0           187.0
70      11.0           184.0
     shop_id  price_per_item
1       92.0            90.0
124      2.0            94.0
216     32.0           101.0
13     100.0           111.0
0       53.0           112.0
33       7.0           112.0
162     93.0           114.0
73      14.0           116.0
82      48.0           117.0
86      56.0           117.0


Checking the top 10 highest priced shoes and last 10 lowest priced shoes we find that the lowest 10 and top 9 are all priced less than $1000. 
However there is only shoe which is very costly
**shoe at shop_id 78**


In [112]:
anamoly_shop=df.loc[df['shop_id']==78]
print(len(anamoly_shop))
print(len(anamoly_shop.loc[anamoly_shop["price_per_item"]==25725]))

46
46


For every order that has been made from shop_id 78 the price is 25725 per shoe that is the error is not for a few observation. The possible reason maybe the shoe price has been entered in a lower denomination like cents. 


**The average order value can be recalculated as modifying the price of the item or removing the orders from the shop 78**


In [113]:
new_avg_order=df[["price_per_item","total_items"]].copy()
updatedfield=new_avg_order["price_per_item"]!=25725
updatedfield
new_avg_order=new_avg_order[updatedfield]
new_avg_order["total_amount"]=new_avg_order["total_items"]*new_avg_order["price_per_item"]
new_avg_order

#total_avg=pd.DataFrame(new_avg_order["total_items"]*new_avg_order["price_per_item"])

,price_per_item,total_items,total_amount
0,112.0,2.0,224.0
1,90.0,1.0,90.0
2,144.0,1.0,144.0
3,156.0,1.0,156.0
4,156.0,1.0,156.0
...,...,...,...
4995,165.0,2.0,330.0
4996,117.0,2.0,234.0
4997,117.0,3.0,351.0
4998,177.0,2.0,354.0


So we have created a new data frame where we have removed the outlier of shop id 78. Doing this we are left with only 4954 rows.
and there were 46 rows in with id 78

In [114]:
n=len(new_avg_order)
print(n)
total_final1=new_avg_order["total_amount"].sum()
print(total_final1)
avg_final1=total_final1/n
avg_final1

4954
13461840.0


2717.3677836092047

**So the average value is $2717.37 when the shop 78 is left out**

In [106]:
new_avg_order2=df[["price_per_item","total_items"]].copy()
updatedfield=new_avg_order2["price_per_item"]==25725
updatedfield
new_avg_order=new_avg_order[updatedfield]
new_avg_order2["total_amount"]=new_avg_order2["total_items"]*new_avg_order2["price_per_item"]
new_avg_order2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,price_per_item,total_items,total_amount
0,112.0,2.0,224.0
1,90.0,1.0,90.0
2,144.0,1.0,144.0
3,156.0,1.0,156.0
4,156.0,1.0,156.0
...,...,...,...
4995,165.0,2.0,330.0
4996,117.0,2.0,234.0
4997,117.0,3.0,351.0
4998,177.0,2.0,354.0


Calculating the avg for the 46 orders

In [108]:

n=len(new_avg_order2)
print(n)
total_final2=new_avg_order2["total_amount"].sum()
print(total_final2)
avg_final2=total_final2/(n*100)
avg_final2

5000
15725640.0


31.45128

In [98]:
avg_final2=(avg_final2*46+avg_final1*4954)/5000 # using mean= (X1'*count(x1) +X2'*count(x2))/(count(x1+x2))

In [99]:
avg_final2

2696.8956

**So on reducing the value of 25725 cents to $257.25**


We get an average of 2696.9

Since the dataset is partially skewed with a few outliers median can be used to  the dataset since the rest value is uniformly distributed

In [121]:
new_avg_order["total_amount"].median()

284.0

The median of the rectified dataset is 284